In [2]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image
from typing import Dict, Tuple
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def euclidean_distance_3d(p1, p2) -> float:
    """Calculate 3D Euclidean distance in meters."""
    return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2 + (p1.z - p2.z)**2)


def euclidean_distance_2d(p1, p2) -> float:
    """Calculate 2D Euclidean distance."""
    return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)


class Landmark3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z


def estimate_body_measurements(pil_image: Image.Image, user_height_cm: float = 170) -> Dict:
    """
    Estimates body measurements (waist, chest, hip) from a single image using pose estimation.
    
    Args:
        pil_image: PIL Image of a person standing upright
        user_height_cm: User's actual height in centimeters for scaling
    
    Returns:
        Dictionary containing measurements and annotated image
    """
    image_rgb = np.array(pil_image.convert("RGB"))
    output_image = image_rgb.copy()
    height, width, _ = output_image.shape
    
    with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=False,
        min_detection_confidence=0.5
    ) as pose:
        results = pose.process(image_rgb)
        
        if not results.pose_landmarks or not results.pose_world_landmarks:
            raise ValueError("No pose detected. Please ensure the person is clearly visible and standing upright.")
        
        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            image=output_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2)
        )
        
        # Get landmark accessor functions
        world_landmarks = results.pose_world_landmarks.landmark
        pixel_landmarks = results.pose_landmarks.landmark
        
        def get_world(name): 
            return world_landmarks[mp_pose.PoseLandmark[name].value]
        
        def get_pixel(name): 
            return pixel_landmarks[mp_pose.PoseLandmark[name].value]
        
        # Calculate body height from pose for scaling
        head_top = get_world("NOSE")  # Approximate head top
        left_ankle = get_world("LEFT_ANKLE")
        right_ankle = get_world("RIGHT_ANKLE")
        
        # Use average of both ankles for ground reference
        ground_y = (left_ankle.y + right_ankle.y) / 2
        pose_height_m = abs(head_top.y - ground_y)
        
        # Scaling factor to convert pose measurements to real-world measurements
        scale_factor = user_height_cm / (pose_height_m * 100)  # Convert to cm scale
        
        # === CHEST MEASUREMENT ===
        left_shoulder = get_world("LEFT_SHOULDER")
        right_shoulder = get_world("RIGHT_SHOULDER")
        
        # Get elbow landmarks to better estimate chest/bust area
        try:
            left_elbow = get_world("LEFT_ELBOW")
            right_elbow = get_world("RIGHT_ELBOW")
            # Use elbow x-coordinates for chest width (arms naturally at chest level)
            chest_width_m = abs(left_elbow.x - right_elbow.x)
        except:
            # Fallback to shoulder-based measurement
            chest_width_m = euclidean_distance_3d(left_shoulder, right_shoulder)
        
        # More conservative chest circumference calculation
        # Chest is typically 1.8-2.2 times the width measurement
        chest_circumference_cm = chest_width_m * 100 * scale_factor * 2.0
        
        # === WAIST MEASUREMENT ===
        left_hip = get_world("LEFT_HIP")
        right_hip = get_world("RIGHT_HIP")
        
        # Waist is typically at the narrowest point between chest and hips
        waist_left = Landmark3D(
            x=(left_shoulder.x + left_hip.x) / 2 * 0.9,  # Narrower than shoulders/hips
            y=(left_shoulder.y + left_hip.y) / 2,
            z=(left_shoulder.z + left_hip.z) / 2
        )
        waist_right = Landmark3D(
            x=(right_shoulder.x + right_hip.x) / 2 * 0.9,
            y=(right_shoulder.y + right_hip.y) / 2,
            z=(right_shoulder.z + right_hip.z) / 2
        )
        
        # More accurate waist calculation using hip and shoulder reference
        waist_width_m = euclidean_distance_3d(waist_left, waist_right)
        waist_circumference_cm = waist_width_m * 100 * scale_factor * 2.5  # More realistic multiplier
        
        # === HIP MEASUREMENT ===
        hip_width_m = euclidean_distance_3d(left_hip, right_hip)
        hip_circumference_cm = hip_width_m * 100 * scale_factor * 2.2  # More realistic multiplier
        
        # Draw measurement points
        def project_to_pixel(world_lm):
            """Convert world landmark to pixel coordinates"""
            return int(world_lm.x * width + width/2), int(-world_lm.y * height + height/2)
        
        # Mark key measurement points including elbows for chest reference
        measurement_points = [
            ("LEFT_SHOULDER", left_shoulder, (255, 0, 0)),
            ("RIGHT_SHOULDER", right_shoulder, (255, 0, 0)),
            ("LEFT_HIP", left_hip, (0, 255, 0)),
            ("RIGHT_HIP", right_hip, (0, 255, 0)),
        ]
        
        # Add elbow points if available for chest measurement verification
        try:
            left_elbow = get_world("LEFT_ELBOW")
            right_elbow = get_world("RIGHT_ELBOW")
            measurement_points.extend([
                ("LEFT_ELBOW", left_elbow, (255, 255, 0)),
                ("RIGHT_ELBOW", right_elbow, (255, 255, 0))
            ])
        except:
            pass
        
        for name, landmark, color in measurement_points:
            x, y = project_to_pixel(landmark)
            # Clamp coordinates to image bounds
            x = max(0, min(width-1, x))
            y = max(0, min(height-1, y))
            cv2.circle(output_image, (x, y), 8, color, -1)
            cv2.putText(output_image, name.replace("_", " "), (x + 10, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Add measurement text to image
        measurements_text = [
            f"Chest: {chest_circumference_cm:.1f} cm",
            f"Waist: {waist_circumference_cm:.1f} cm",
            f"Hip: {hip_circumference_cm:.1f} cm",
            f"Height: {user_height_cm} cm (input)"
        ]
        
        for i, text in enumerate(measurements_text):
            cv2.putText(output_image, text, (20, 40 + i * 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(output_image, text, (20, 40 + i * 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1)
        
        return {
            "annotated_image": Image.fromarray(output_image),
            "measurements_cm": {
                "chest_circumference_cm": round(chest_circumference_cm, 1),
                "waist_circumference_cm": round(waist_circumference_cm, 1),
                "hip_circumference_cm": round(hip_circumference_cm, 1)
            },
            "debug_info": {
                "pose_height_m": round(pose_height_m, 3),
                "scale_factor": round(scale_factor, 3),
                "chest_width_m": round(chest_width_m, 4),
                "waist_width_m": round(waist_width_m, 4),
                "hip_width_m": round(hip_width_m, 4)
            }
        }


# Usage example
def main():
    # Constants
    user_height_cm = 170  # Replace with actual user height
    image_path = "../images/front-image.png"  # Replace with your image path
    
    try:
        # Load image
        img = Image.open(image_path)
        
        # Get measurements
        result = estimate_body_measurements(img, user_height_cm)
        
        # Display results
        result["annotated_image"].show()
        
        measurements = result["measurements_cm"]
        print("\n=== BODY MEASUREMENTS ===")
        print(f"Chest Circumference: {measurements['chest_circumference_cm']} cm")
        print(f"Waist Circumference: {measurements['waist_circumference_cm']} cm")
        print(f"Hip Circumference: {measurements['hip_circumference_cm']} cm")
        
        print("\n=== DEBUG INFO ===")
        debug = result["debug_info"]
        print(f"Detected pose height: {debug['pose_height_m']} m")
        print(f"Scale factor: {debug['scale_factor']}")
        
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure:")
        print("1. The image path is correct")
        print("2. The person is clearly visible and standing upright")
        print("3. The person is facing the camera")


if __name__ == "__main__":
    main()


=== BODY MEASUREMENTS ===
Chest Circumference: 103.6 cm
Waist Circumference: 72.2 cm
Hip Circumference: 56.0 cm

=== DEBUG INFO ===
Detected pose height: 1.357 m
Scale factor: 1.253
